In [1]:
from sys import stdout
import numpy as np
from matplotlib import pyplot as plt
import mdtraj
import openmm as mm
from openmm.app import *
from openmm import unit
from openmm.app.simulation import Simulation
from AIGLE.utilities import *
from AIGLE.plugins import GLEIntegrator
from AIGLE import Trajectory

np.set_printoptions(precision=12)


# Parameters
nmain = 20
k_main = 1000.0 * unit.kilojoule_per_mole / unit.nanometer**2   # displacement ~0.5A
d_main = 0.3 
temp = 300 * unit.kelvin
kb = 8.314462618e-3 * unit.kilojoule_per_mole / unit.kelvin
kbT = kb * temp

## setup openmm system
nm = unit.nanometer
mass=np.ones(nmain) * 12 *  unit.amu
# Create a system
system = mm.System()
topol = topology.Topology()
chain = topol.addChain()
init_positions = []
indices_main = []

## Add particles to the system
for ii in range(nmain):
    idx=system.addParticle(mass[ii])
    indices_main.append(idx)
    pos = np.array([ii , 0,0]) * d_main
    init_positions.append(pos)
    residue = topol.addResidue('R{}'.format(ii), chain)
    topol.addAtom('C{}'.format(ii), Element.getBySymbol('C'), residue) 

init_positions = np.array(init_positions) * nm

## Add forces
custom_force  = mm.openmm.HarmonicBondForce()
for ii in range(nmain-1):
    custom_force.addBond( ii, (ii+1), d_main * nm, k_main)
    topol.addBond( ii,  (ii+1) )
system.addForce(custom_force )

0

# simulate GLE

In [ ]:
## load GLE configurations
import json
with open('gle_paras/model_iter_9000.json',) as f:
    gle_config = json.load(f) 
    
dt = 0.01   #timestep [ps]
log_freq = int(0.1/dt)   ## dump frequency
print_freq = int(1000/dt)
integrator = GLEIntegrator(gle_config, dt)


## MD setup
simulation = Simulation(topol, system, integrator)
simulation.context.setPositions(init_positions)

## relax 
simulation.step(100 / dt)

## run
simulation.reporters = [
    mdtraj.reporters.DCDReporter("gle_traj_N{}.dcd".format(nmain), log_freq),
    StateDataReporter(stdout, print_freq, step=True, time=True, temperature=True, elapsedTime=True),
]

total_time = 100 #ns
simulation.step(int(total_time*1000 /dt))

# simulate LE 

In [ ]:
## get the LE integrator
le_integrator = integrator.get_langevin_integrator(temp, dt )

## MD setup
le_simulation = Simulation(topol, system, le_integrator)
le_simulation.context.setPositions(init_positions)

## relax 
le_simulation.step(100 / dt)

## run
le_simulation.reporters = [
    mdtraj.reporters.DCDReporter("le_traj_N{}.dcd".format(nmain), log_freq),
    StateDataReporter(stdout, print_freq, step=True, time=True, temperature=True, elapsedTime=True),
]

total_time = 100 #ns
le_simulation.step(int(total_time*1000 /dt))